### single model pipeline

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
###################################################################
# The dataset is a single channel eeg dataset for the motor task of 
# moving the eyes to the left and rest, alternating every 5 seconds, 
# for a total duration of 600 seconds. 
# Column 1 is index and Column 2 is the signal -> ['idx', 'x'].
###################################################################

df = pd.read_csv("dataset/jg_rest_eye_left_600.csv", index_col=None)
df.head()
df.shape

(6097233, 2)

In [3]:

# Calculate the number of rows and create labels
num_rows = df.shape[0]
num_labels = (num_rows // 50000) + 1  # Calculate the number of labels needed
labels = [i % 2 for i in range(num_labels) for _ in range(50000)]
df['Label'] = labels[:num_rows]
df.to_csv('dataset/labeled_jg_rest_eyeleft_600.csv', index=False)

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import entropy

def extract_features(signal_window):
    # Extract features from the signal window
    features = []
    
    # Basic statistical features
    mean_value = np.mean(signal_window)
    variance = np.var(signal_window)
    min_value = np.min(signal_window)
    max_value = np.max(signal_window)
    range_value = max_value - min_value
    median_value = np.median(signal_window)
    rms = np.sqrt(np.mean(np.square(signal_window)))
    
    # Frequency-domain features (using FFT)
    fft_values = np.fft.fft(signal_window)
    power_spectrum = np.abs(fft_values)**2
    total_power = np.sum(power_spectrum)
    spectral_entropy = entropy(power_spectrum, base=2)
    
    # Other features
    zero_crossings = np.sum(np.diff(np.sign(signal_window)) != 0)
    
    # Add features to the list
    features.extend([mean_value, variance, min_value, max_value,
                     range_value, median_value, rms, total_power, spectral_entropy,
                     zero_crossings])
    
    return features


In [5]:
def label_data(num_windows):
    return [i % 2 for i in range(num_windows)]

In [6]:
# EEG data
eeg_data = df['x']  
window_size = 50000  # 50,000 samples per window
eeg_data.shape


(6097233,)

In [7]:
# Process each window
num_windows = len(eeg_data) // window_size
features = []
labels = []
for i in range(num_windows):
    signal_window = eeg_data[i * window_size: (i + 1) * window_size]
    features.append(extract_features(signal_window))
    # plot_and_save_signal_window(signal_window, i)
labels = label_data(num_windows)

In [8]:
len(features)


121

In [9]:
len(labels)

121

In [10]:
df = pd.DataFrame(features, columns=['mean_value', 'variance', 'min_value', 'max_value',
                     'range_value', 'median_value', 'rms', 'total_power', 'spectral_entropy',
                     'zero_crossings'])
df['Label'] = labels


In [11]:
features[99]

[166.5499,
 1926714.82778999,
 -2984,
 4019,
 7003,
 253.0,
 1398.0177742003139,
 4886134242450000.0,
 5.973377899601365,
 70]

In [12]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [13]:
len(y_test)

25

In [14]:

len(y_train)

96

In [15]:
# Train SVM classifier
model = SVC(random_state=42)
model.fit(X_train, y_train)



SVC(random_state=42)

In [16]:
# Evaluate the classifier
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.96


In [17]:
len(X_test)

25

In [18]:
######################### END of SVM ########################################